<a href="https://colab.research.google.com/github/HrithikNambiar/COVID-19-Cough-Detection/blob/main/ds_featureextractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
cd '/content/drive/MyDrive/ComParE2021'

/content/drive/MyDrive/ComParE2021


In [ ]:
!ls

activate	  chunk4.wav	 features			   README.md
audeep_workspace  chunk5.wav	 features_2			   results
augmented_dist	  chunk6.wav	 Miniconda3-4.5.4-Linux-x86_64.sh  scripts
chunk0.wav	  data		 notebooks			   smile.log
chunk1.wav	  data_2	 opensmile			   src
chunk2.wav	  dist		 openxbow
chunk3.wav	  end2you_files  params.yaml


In [ ]:
!pip install -U PyYAML
!pip install liac-arff
!pip install -U scikit-learn scipy matplotlib

In [ ]:
pip install deepspectrum

In [ ]:
import os, yaml, subprocess, glob
verbose_option='-v'

In [ ]:
params = {}
with open('params.yaml') as f:
    params = yaml.load(f, Loader=yaml.FullLoader)
    params = params['deepspectrum']


# base directory for audio files
audio_base='/content/drive/MyDrive/ComParE2021/data_2/train'
output_base='/content/drive/MyDrive/ComParE2021/features_2/deepspectrum'
label_base='/content/drive/MyDrive/ComParE2021/data_2'
label_files = glob.glob(f'{label_base}/*.csv')


In [ ]:
if not os.path.exists(output_base):
    os.makedirs(output_base)


# plot mode: mel, spectrogram, chroma
plotMode=params['plotMode']

# if mode == mel --> number of mel bands
melBands=params['melBands']

# good choices can be "magma", "plasma", "viridis" or "cividis"
colourMap = params['colourMap']

# if windoing
window_size = params['window_size']
hop_size = params['hop_size']

In [ ]:
# DeepSpectrum supports the following pre-trained CNN networks: vgg16, vgg19, resnet50, inception_resnet_v2,
# xception, densenet121, densenet169, densenet201, mobilenet, mobilenet_v2, nasnet_large, nasnet_mobile, alexnet,
# squeezenet, googlenet.
extractionNetwork = params['extractionNetwork']
# recommended feature layer for all pre-trained CNNs except for vgg16, vgg19, and alexnet: "avg_pool"
# recommended feature layer for vgg16, vgg19, and alexnet: "fc2"
for network in extractionNetwork:
    if network == 'vgg16' or network == 'vgg19' or network == 'alexnet':
        featureLayer = 'fc2'
    else:
        featureLayer = 'avg_pool'   
    for mode in plotMode:
        if mode == 'mel':
            for nmel in melBands:
                for colour in colourMap:
                    for label_file in label_files:
                        fid = [network, featureLayer, mode, str(nmel), colour, os.path.basename(label_file)]
                        featFile = os.path.join(*fid)
                        outputFeat = os.path.join(output_base, featFile)
                        cmd='deepspectrum ' + verbose_option + ' features ' + os.path.join(audio_base) \
                            + ' -o ' + outputFeat + ' -en ' + network \
                            + ' -fl ' + featureLayer + ' -m ' + mode + ' -nm ' + str(nmel) \
                            + ' -cm ' + colour + ' -lf ' + label_file 
                        if window_size is not None:
                            cmd += ' -t ' + str(window_size) + ' ' + str(hop_size)
                        print(cmd)
                        os.system(cmd)
                    # subprocess.Popen(cmd, shell=True).wait()
        elif mode == 'spectrogram' or mode == 'chroma':
            for colour in colourMap:
                for label_file in label_files:
                    fid = [network, featureLayer, mode, colour, os.path.basename(label_file)]
                    featFile = os.path.join(*fid)
                    outputFeat = os.path.join(output_base, featFile)
                    cmd='deepspectrum ' + verbose_option + ' features ' + os.path.join(audio_base) \
                        + ' -o ' + outputFeat + ' -en ' + network \
                        + ' -fl ' + featureLayer + ' -m ' + mode \
                        + ' -cm ' + colour + ' -lf ' + label_file 
                    if window_size is not None:
                        cmd += ' -t ' + str(window_size) + ' ' + str(hop_size)
                    print(cmd)
                    os.system(cmd)
                # subprocess.Popen(cmd, shell=True).wait()

deepspectrum -v features /content/drive/MyDrive/ComParE2021/data_2/train -o /content/drive/MyDrive/ComParE2021/features_2/deepspectrum/densenet121/avg_pool/mel/128/viridis/train.csv -en densenet121 -fl avg_pool -m mel -nm 128 -cm viridis -lf /content/drive/MyDrive/ComParE2021/data_2/train.csv
deepspectrum -v features /content/drive/MyDrive/ComParE2021/data_2/train -o /content/drive/MyDrive/ComParE2021/features_2/deepspectrum/densenet121/avg_pool/mel/128/viridis/test.csv -en densenet121 -fl avg_pool -m mel -nm 128 -cm viridis -lf /content/drive/MyDrive/ComParE2021/data_2/test.csv
deepspectrum -v features /content/drive/MyDrive/ComParE2021/data_2/train -o /content/drive/MyDrive/ComParE2021/features_2/deepspectrum/densenet121/avg_pool/mel/128/viridis/val.csv -en densenet121 -fl avg_pool -m mel -nm 128 -cm viridis -lf /content/drive/MyDrive/ComParE2021/data_2/val.csv


In [ ]:
!python -m src.svm deepspectrum

Fitting 1 folds for each of 20 candidates, totalling 20 fits
UAR: 0.6362704918032787
              precision    recall  f1-score   support

    negative       0.85      0.81      0.83       183
    positive       0.39      0.46      0.42        48

    accuracy                           0.74       231
   macro avg       0.62      0.64      0.63       231
weighted avg       0.76      0.74      0.75       231


Confusion Matrix:

[[149  34]
 [ 26  22]]
Generating test predictions for optimised parameters {'estimator': "LinearSVC(C=0.1, class_weight='balanced', max_iter=10000, random_state=42)", 'estimator__C': '0.1', 'estimator__class_weight': 'balanced', 'estimator__loss': 'squared_hinge', 'estimator__max_iter': 10000, 'scaler': 'None'}


In [ ]:
!python -m src.svm deepspectrum

Fitting 1 folds for each of 20 candidates, totalling 20 fits
UAR: 0.6466871584699454
              precision    recall  f1-score   support

    negative       0.86      0.81      0.83       183
    positive       0.40      0.48      0.44        48

    accuracy                           0.74       231
   macro avg       0.63      0.65      0.64       231
weighted avg       0.76      0.74      0.75       231


Confusion Matrix:

[[149  34]
 [ 25  23]]
Generating test predictions for optimised parameters {'estimator': "LinearSVC(C=1e-05, class_weight='balanced', max_iter=10000, random_state=42)", 'estimator__C': '1e-05', 'estimator__class_weight': 'balanced', 'estimator__loss': 'squared_hinge', 'estimator__max_iter': 10000, 'scaler': 'None'}
